In [ ]:
### this example demonstrates how to use kfp python sdk to create pipeline / update its versions and trigger runs
### to see full doc on how we can use python sdk, please refer to: 
### https://kubeflow-pipelines.readthedocs.io/en/stable/index.html

In [ ]:
with open("requirements.txt", "w") as f:
    f.write("python-dateutil\n")
    f.write("kubernetes\n")
    f.write("kubeflow-fairing\n")
    f.write("kfp==1.4.0\n")
    
!pip install -r requirements.txt  --upgrade --user

In [14]:
# as codes are executed on kernel of jupyter, a pod on kubernetes cluster
# so it should use internal dns for looking up global entry from istio-system
host = 'http://istio-ingressgateway.istio-system.svc.cluster.local:80/pipeline'
namespace = 'kubeflow-user'

### update your authservice_session everytime when you logged into kubeflow.
### trigger your developer console and to go application tab, you can see cookies section.
authservice_session = 'authservice_session=MTYxODM2ODYwMXxOd3dBTkVOUVRFNVVOa0pCUmpWRE4wazNURU15UTA0eldscExWRTFaU1ZOYVFWZEdSRlUwU1VwQ1ZUVkZVVVZFTjFwQ1ZqUlJWRkU9fFSCMXFfH2HQDF6qrk9rKvMoUTXhKwH1z0Ps_VZJODI8'

In [41]:
import kfp
import json
import logging

client = kfp.Client(host=host, cookies=authservice_session)
list_exps = client.list_experiments(namespace=namespace)
print(list_exps)

{'experiments': [{'created_at': datetime.datetime(2021, 4, 6, 9, 7, 56, tzinfo=tzlocal()),
                  'description': None,
                  'id': '8bedfc1e-139c-497c-b4e6-513d46d75f5f',
                  'name': 'test exp',
                  'resource_references': [{'key': {'id': 'kubeflow-user',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}


In [42]:
pipeline_template_path = "helloworld.zip"
pipeline_name = "helloworld"
pipeline_description = "some descriptions..."

In [43]:
# create an pipeline with name/description/template local path

upload_pipeline = client.pipeline_uploads.upload_pipeline(pipeline_template_path,
    name=pipeline_name,
    description=pipeline_description,
    async_req=False,
)
print(upload_pipeline)

{'created_at': datetime.datetime(2021, 4, 14, 4, 15, 34, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2021, 4, 14, 4, 15, 34, tzinfo=tzlocal()),
                     'id': 'e59bf39c-7da6-4831-be24-54952eddc934',
                     'name': 'helloworld',
                     'package_url': None,
                     'parameters': [{'name': 'text1', 'value': 'message 1'},
                                    {'name': 'text2', 'value': 'message 2'}],
                     'resource_references': [{'key': {'id': 'e59bf39c-7da6-4831-be24-54952eddc934',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': 'some descriptions...',
 'error': None,
 'id': 'e59bf39c-7da6-4831-be24-54952eddc934',
 'name': 'helloworld',
 'parameters': [{'name': 'text1', 'value

In [46]:
# update pipeline version
pipeline_id=upload_pipeline.id

import datetime
timed_version = datetime.datetime.today().strftime("%d/%m/%Y %H:%M:%S")

upload_pipeline_version = client.pipeline_uploads.upload_pipeline_version(pipeline_template_path,
    name=timed_version,
    pipelineid=pipeline_id,
    async_req=False,
)
print(upload_pipeline_version)

{'code_source_url': None,
 'created_at': datetime.datetime(2021, 4, 14, 4, 17, 1, tzinfo=tzlocal()),
 'id': '3d3ec59b-a31b-49f7-8e61-0edb3d359ef0',
 'name': '14/04/2021 04:17:01',
 'package_url': None,
 'parameters': [{'name': 'text1', 'value': 'message 1'},
                {'name': 'text2', 'value': 'message 2'}],
 'resource_references': [{'key': {'id': 'e59bf39c-7da6-4831-be24-54952eddc934',
                                  'type': 'PIPELINE'},
                          'name': None,
                          'relationship': 'OWNER'}]}


In [47]:
### trigger run on a specific pipeline
experimentid=list_exps.experiments[0].id
versionid=upload_pipeline_version.id
job_name = "run helloworld"

run_resp = client.run_pipeline(
    experimentid,
    job_name,
    pipeline_id=pipeline_id,
    version_id=versionid,
)
print(run_resp)

{'created_at': datetime.datetime(2021, 4, 14, 4, 18, 9, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
 'id': 'f022ba66-4d12-42eb-974a-140a60e8ac34',
 'metrics': None,
 'name': 'run helloworld',
 'pipeline_spec': {'parameters': None,
                   'pipeline_id': 'e59bf39c-7da6-4831-be24-54952eddc934',
                   'pipeline_manifest': None,
                   'pipeline_name': 'helloworld',
                   'workflow_manifest': '{"kind":"Workflow","apiVersion":"argoproj.io/v1alpha1","metadata":{"generateName":"execution-order-pipeline-","creationTimestamp":null,"labels":{"pipelines.kubeflow.org/kfp_sdk_version":"1.4.0"},"annotations":{"pipelines.kubeflow.org/kfp_sdk_version":"1.4.0","pipelines.kubeflow.org/pipeline_compilation_time":"2021-04-13T07:02:18.905505","pipelines.kubeflow.org/pipeline_spec":"{\\"description\\": '
                                        '\\"A pipeline to demonstrate '
 